In [19]:
import pandas as pd
from urllib.parse import urlparse
from pyspark.sql import SparkSession

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType

In [2]:
spark = SparkSession.builder \
    .appName("jslibs-top20") \
    .master("spark://spark-master:7077") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "file:///opt/spark/spark-events/") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", 1) \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.dynamicAllocation.shuffleTracking.enabled", "true") \
    .config("spark.dynamicAllocation.initialExecutors", 2) \
    .config("spark.dynamicAllocation.minExecutors", 2) \
    .config("spark.dynamicAllocation.maxExecutors", 15) \
    .config("spark.scheduler.mode", "FAIR") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/14 08:44:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/14 08:44:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/14 08:44:36 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.


In [4]:
spark_df = spark.read.parquet('/opt/workspace/warc_yearly/script_extraction_out_2024/')
df_rows = spark_df.count()
df_cols = len(spark_df.columns)
(df_rows, df_cols)

(22127510, 5)

In [5]:
spark_df.show()

+--------------+--------------------+--------------------+--------------------+--------------------+
|            ip|                host|              server|              emails|    script_src_attrs|
+--------------+--------------------+--------------------+--------------------+--------------------+
| 38.165.50.153|http://04v.career...|               nginx|mailto:al_andino@...|http://kit.fontaw...|
| 159.75.83.151|http://088022.com...|Apache/2.4.39 (Wi...|                    |http://088022.com...|
| 38.165.49.137|http://1.vapemanz...|               nginx|                    |http://query.aliy...|
|178.254.10.206|http://118527.web...|              Apache|                    |./styles/pronight...|
|219.234.30.122|http://1395656471...|           wts/1.7.4|                    |/pc/js/jquery-1.1...|
|  173.231.4.85|http://14star.com...|Apache/2.2.31 (Un...|mailto:www14star@...|include/js/common...|
| 159.69.237.22|http://15938.home...|               nginx|mailto:emilyy.rod...|//img.homepa

In [15]:
sample_host = spark_df.first()['host']
urlparse(sample_host).netloc

'04v.career-bengoshi.net'

In [17]:
domain_parser = spark.sparkContext.broadcast(urlparse)

@udf(StringType())
def get_domain(host_url):
    parser = domain_parser.value
    parsed_url = parser(host_url)
    return parsed_url.netloc

In [18]:
spark_df_domains = spark_df.withColumn('domain', get_domain('host'))
spark_df_domains.show()

+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            ip|                host|              server|              emails|    script_src_attrs|              domain|
+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 38.165.50.153|http://04v.career...|               nginx|mailto:al_andino@...|http://kit.fontaw...|04v.career-bengos...|
| 159.75.83.151|http://088022.com...|Apache/2.4.39 (Wi...|                    |http://088022.com...|          088022.com|
| 38.165.49.137|http://1.vapemanz...|               nginx|                    |http://query.aliy...|    1.vapemanzil.com|
|178.254.10.206|http://118527.web...|              Apache|                    |./styles/pronight...|118527.webhosting...|
|219.234.30.122|http://1395656471...|           wts/1.7.4|                    |/pc/js/jquery-1.1...|     13956564713.com|
|  173.231.4.85|http://1

# tiktoken

In [ ]:
import tiktoken